In [43]:
import numpy as np
from PIL import Image
import cv2 #opencv
import io
import time
import pandas as pd
import numpy as np
from IPython.display import clear_output
from random import randint
import os

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

#keras imports
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD , Adam
from keras.callbacks import TensorBoard
from collections import deque
import random
import pickle
from io import BytesIO
import base64
import json
import tensorflow as tf
 
print("VERSION", tf.Session(config=tf.ConfigProto(log_device_placement=True)))

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

VERSION <tensorflow.python.client.session.Session object at 0x0000016A8694D198>
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3727641847237263452
]


In [44]:
#path variables
game_url = "http://flappybird.io/"
chrome_driver_path = "./chromedriver"
loss_file_path = "./objects/loss_df.csv"
actions_file_path = "./objects/actions_df.csv"
q_value_file_path = "./objects/q_values.csv"
scores_file_path = "./objects/scores_df.csv"

#scripts
#create id for canvas for faster selection from DOM
init_script = "document.getElementsByClassName('canvas-container')[0].id = 'canvas-container'"

#get image from canvas
# getbase64Script = "canvasContainer = document.getElementById('canvas-container'); \
# return canvasContainer.toDataURL().substring(22)"
getbase64Script = "return document.getElementById('testCanvas').toDataURL().substring(22)"

alive_t0 = 0
alive_tf = 0

In [45]:
'''
* Game class: Selenium interfacing between the python and browser
* __init__():  Launch the broswer window using the attributes in chrome_options
* get_crashed() : return true if the agent as crashed on an obstacles. Gets javascript variable from game decribing the state
* get_playing(): true if game in progress, false is crashed or paused
* restart() : sends a signal to browser-javascript to restart the game
* press_up(): sends a single to press up get to the browser
* get_score(): gets current game score from javascript variables.
* pause(): pause the game
* resume(): resume a paused game if not crashed
* end(): close the browser and end the game
'''
class Game:
    def __init__(self,custom_config=True):
        chrome_options = Options()
        chrome_options.add_argument("disable-infobars")
        chrome_options.add_argument("--mute-audio")
        self._driver = webdriver.Chrome(executable_path = chrome_driver_path,chrome_options=chrome_options)
        self._driver.set_window_position(x=-10,y=0)
        self._driver.get(game_url)
        # self._driver.execute_script("Runner.config.ACCELERATION=0")
        self._driver.execute_script(init_script)
    def get_crashed(self):
        return self._driver.execute_script("return dead")
    def get_playing(self):
        return self._driver.execute_script("return started")
    def restart(self):
        global alive_t0
        self._driver.execute_script("spacebar();")
        alive_t0 = int(round(time.time() * 1000))
    def press_up(self):
        self._driver.execute_script("spacebar();")
    def get_score(self):
        global alive_t0
        global alive_tf
        alive_tf = int(round(time.time() * 1000))
        score = alive_tf - alive_t0
#         score = ''.join(score_array) # the javascript object is of type array with score in the formate[1,0,0] which is 100.
        return score
    def pause(self):
        return self._driver.execute_script("spacebar();")
    def resume(self):
        return self._driver.execute_script("spacebar();")
    def end(self):
        self._driver.close()

In [46]:
class DinoAgent:
    def __init__(self,game): #takes game as input for taking actions
        self._game = game; 
        self.jump(); #to start the game, we need to jump once
    def is_running(self):
        return self._game.get_playing()
    def is_crashed(self):
        return self._game.get_crashed()
    def jump(self):
        self._game.press_up()
    def duck(self):
        self._game.press_down()

In [ ]:
class Game_sate:
    def __init__(self,agent,game):
        self._agent = agent
        self._game = game
        self._display = show_img() #display the processed image on screen using openCV, implemented using python coroutine 
        self._display.__next__() # initiliaze the display coroutine 
    def get_state(self,actions):
        actions_df.loc[len(actions_df)] = actions[1] # storing actions in a dataframe
        score = self._game.get_score() 
        reward = 0.1
        is_over = False #game over
        if actions[1] == 1:
            self._agent.jump()
        image = grab_screen(self._game._driver) 
        self._display.send(image) #display the image on screen
        if self._agent.is_crashed():
            scores_df.loc[len(loss_df)] = score # log the score when game is over
            self._game.restart()
            reward = -1
            is_over = True
        return image, reward, is_over #return the Experience tuple

In [ ]:
def save_obj(obj, name ):
    with open('objects/'+ name + '.pkl', 'wb') as f: #dump files into objects folder
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_obj(name ):
    with open('objects/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def grab_screen(_driver):
    image_b64 = _driver.execute_script(getbase64Script)
    screen = np.array(Image.open(BytesIO(base64.b64decode(image_b64))))
    image = process_img(screen)#processing image as required
    return image

def process_img(image):
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #RGB to Grey Scale
    image = image[:1200, :1000] #Crop Region of Interest(ROI)
    image = cv2.resize(image, (80,80))
    return  image

def show_img(graphs = False):
    """
    Show images in new window
    """
    while True:
        screen = (yield)
        window_title = "logs" if graphs else "game_play"
        cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)        
        imS = cv2.resize(screen, (800, 400)) 
        cv2.imshow(window_title, screen)
        if (cv2.waitKey(1) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break

In [ ]:
#Intialize log structures from file if exists else create new
loss_df = pd.read_csv(loss_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns =['loss'])
scores_df = pd.read_csv(scores_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns = ['scores'])
actions_df = pd.read_csv(actions_file_path) if os.path.isfile(actions_file_path) else pd.DataFrame(columns = ['actions'])
q_values_df =pd.read_csv(actions_file_path) if os.path.isfile(q_value_file_path) else pd.DataFrame(columns = ['qvalues'])

In [ ]:
#game parameters
ACTIONS = 2 # possible actions: jump, do nothing
GAMMA = 0.99 # decay rate of past observations original 0.99
OBSERVATION = 100. # timesteps to observe before training
EXPLORE = 100000  # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 16 # size of minibatch
FRAME_PER_ACTION = 1
LEARNING_RATE = 1e-4
img_rows , img_cols = 80,80
img_channels = 4 #We stack 4 frames

In [ ]:
# training variables saved as checkpoints to filesystem to resume training from the same step
def init_cache():
    """initial variable caching, done only once"""
    save_obj(INITIAL_EPSILON,"epsilon")
    t = 0
    save_obj(t,"time")
    D = deque()
    save_obj(D,"D")

In [ ]:
'''Call only once to init file structure
'''
#init_cache()

'Call only once to init file structure\n'

In [ ]:
def buildmodel():
    print("Now we build the model")
    model = Sequential()
    model.add(Conv2D(32, (8, 8), padding='same',strides=(4, 4),input_shape=(img_cols,img_rows,img_channels)))  #80*80*4
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (4, 4),strides=(2, 2),  padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3),strides=(1, 1),  padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS))
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    
    #create model file if not present
    if not os.path.isfile(loss_file_path):
        model.save_weights('model.h5')
    print("We finish building the model")
    return model

In [ ]:
''' 
main training module
Parameters:
* model => Keras Model to be trained
* game_state => Game State module with access to game environment and dino
* observe => flag to indicate wherther the model is to be trained(weight updates), else just play
'''
def trainNetwork(model,game_state,observe=False):
    last_time = time.time()
    # store the previous observations in replay memory
    D = load_obj("D") #load from file system
    # get the first state by doing nothing
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] =1 #0 => do nothing,
                     #1=> jump
    
    x_t, r_0, terminal = game_state.get_state(do_nothing) # get next step after performing the action
    

    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2) # stack 4 images to create placeholder input
    

    
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*20*40*4
    
    initial_state = s_t 

    if observe :
        OBSERVE = 999999999    #We keep observe, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        epsilon = load_obj("epsilon") 
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)

    t = load_obj("time") # resume from the previous time step stored in file system
    while (True): #endless running
        
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0 #reward at 4
        a_t = np.zeros([ACTIONS]) # action at t
        
        #choose an action epsilon greedy
        if t % FRAME_PER_ACTION == 0: #parameter to skip frames for actions
            if  random.random() <= epsilon: #randomly explore an action
                print("----------Random Action----------")
                action_index = random.randrange(ACTIONS)
                a_t[action_index] = 1
            else: # predict the output
                q = model.predict(s_t)       #input a stack of 4 images, get the prediction
                max_Q = np.argmax(q)         # chosing index with maximum q value
                action_index = max_Q 
                a_t[action_index] = 1        # o=> do nothing, 1=> jump
                
        #We reduced the epsilon (exploration parameter) gradually
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE 

        #run the selected action and observed next state and reward
        x_t1, r_t, terminal = game_state.get_state(a_t)
        print('fps: {0}'.format(1 / (time.time()-last_time))) # helpful for measuring frame rate
        last_time = time.time()
        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x20x40x1
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3) # append the new image to input stack and remove the first one
        
        
        # store the transition in D
        D.append((s_t, action_index, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        #only train if done observing
        if t > OBSERVE: 
            
            #sample a minibatch to train on
            minibatch = random.sample(D, BATCH)
            inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))   #32, 20, 40, 4
            targets = np.zeros((inputs.shape[0], ACTIONS))                         #32, 2

            #Now we do the experience replay
            for i in range(0, len(minibatch)):
                state_t = minibatch[i][0]    # 4D stack of images
                action_t = minibatch[i][1]   #This is action index
                reward_t = minibatch[i][2]   #reward at state_t due to action_t
                state_t1 = minibatch[i][3]   #next state
                terminal = minibatch[i][4]   #wheather the agent died or survided due the action
                

                inputs[i:i + 1] = state_t    

                targets[i] = model.predict(state_t)  # predicted q values
                Q_sa = model.predict(state_t1)      #predict q values for next step
                
                if terminal:
                    targets[i, action_t] = reward_t # if terminated, only equals reward
                else:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)

            loss += model.train_on_batch(inputs, targets)
            loss_df.loc[len(loss_df)] = loss
            q_values_df.loc[len(q_values_df)] = np.max(Q_sa)
        s_t = initial_state if terminal else s_t1 #reset game to initial frame if terminate
        t = t + 1
        
        # save progress every 1000 iterations
        if t % 1000 == 0:
            print("Now we save model")
            game_state._game.pause() #pause game while saving to filesystem
            model.save_weights("model.h5", overwrite=True)
            save_obj(D,"D") #saving episodes
            save_obj(t,"time") #caching time steps
            save_obj(epsilon,"epsilon") #cache epsilon to avoid repeated randomness in actions
            loss_df.to_csv("./objects/loss_df.csv",index=False)
            scores_df.to_csv("./objects/scores_df.csv",index=False)
            actions_df.to_csv("./objects/actions_df.csv",index=False)
            q_values_df.to_csv(q_value_file_path,index=False)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)
            clear_output()
            game_state._game.resume()
        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", t, "/ STATE", state,             "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t,             "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)

    print("Episode finished!")
    print("************************")


In [ ]:
#main function
def playGame(observe=False):
    game = Game()
    time.sleep(3)
    dino = DinoAgent(game)
    game_state = Game_sate(dino,game)    
    model = buildmodel()
    try:
        trainNetwork(model,game_state,observe=observe)
    except StopIteration:
        game.end()

In [ ]:
playGame(observe=False);

TIMESTEP 55000 / STATE explore / EPSILON 0.045155898999916205 / ACTION 1 / REWARD 0.1 / Q_MAX  0.3143628 / Loss  0.22799313068389893
fps: 0.03689671859759984
TIMESTEP 55001 / STATE explore / EPSILON 0.045154899999916204 / ACTION 0 / REWARD 0.1 / Q_MAX  6.2961807 / Loss  0.2871796488761902
fps: 4.96164174196128
TIMESTEP 55002 / STATE explore / EPSILON 0.0451539009999162 / ACTION 1 / REWARD 0.1 / Q_MAX  8.891499 / Loss  0.09526122361421585
fps: 5.3278390535069855
TIMESTEP 55003 / STATE explore / EPSILON 0.0451529019999162 / ACTION 1 / REWARD 0.1 / Q_MAX  5.167668 / Loss  0.41402798891067505
fps: 5.494752590298548
TIMESTEP 55004 / STATE explore / EPSILON 0.0451519029999162 / ACTION 0 / REWARD 0.1 / Q_MAX  0.75613254 / Loss  0.1992795467376709
fps: 5.384865439475367
TIMESTEP 55005 / STATE explore / EPSILON 0.0451509039999162 / ACTION 1 / REWARD 0.1 / Q_MAX  7.525032 / Loss  0.22685600817203522
fps: 6.065875200120325
TIMESTEP 55006 / STATE explore / EPSILON 0.045149904999916196 / ACTION 1 /

fps: 5.380962216682853
TIMESTEP 55053 / STATE explore / EPSILON 0.045102951999916124 / ACTION 1 / REWARD 0.1 / Q_MAX  6.949543 / Loss  0.19962123036384583
fps: 5.707320325650191
TIMESTEP 55054 / STATE explore / EPSILON 0.04510195299991612 / ACTION 1 / REWARD 0.1 / Q_MAX  2.0771365 / Loss  0.6455889940261841
fps: 5.296921845124887
TIMESTEP 55055 / STATE explore / EPSILON 0.04510095399991612 / ACTION 1 / REWARD 0.1 / Q_MAX  6.645896 / Loss  0.2224184274673462
fps: 4.789472602411235
TIMESTEP 55056 / STATE explore / EPSILON 0.04509995499991612 / ACTION 1 / REWARD 0.1 / Q_MAX  0.4406465 / Loss  0.024462003260850906
fps: 5.355697773600903
TIMESTEP 55057 / STATE explore / EPSILON 0.04509895599991612 / ACTION 1 / REWARD 0.1 / Q_MAX  7.1689262 / Loss  0.15235909819602966
fps: 5.218899158367302
TIMESTEP 55058 / STATE explore / EPSILON 0.04509795699991612 / ACTION 1 / REWARD 0.1 / Q_MAX  6.3873243 / Loss  0.2092328667640686
fps: 5.697706547268926
TIMESTEP 55059 / STATE explore / EPSILON 0.0450969

TIMESTEP 55105 / STATE explore / EPSILON 0.045051003999916045 / ACTION 1 / REWARD 0.1 / Q_MAX  2.2533774 / Loss  0.1290871798992157
fps: 4.369066666666667
TIMESTEP 55106 / STATE explore / EPSILON 0.04505000499991604 / ACTION 0 / REWARD 0.1 / Q_MAX  1.7981222 / Loss  0.11459140479564667
fps: 3.96858290983719
TIMESTEP 55107 / STATE explore / EPSILON 0.04504900599991604 / ACTION 1 / REWARD 0.1 / Q_MAX  7.3638935 / Loss  0.18442988395690918
fps: 4.650752061029767
TIMESTEP 55108 / STATE explore / EPSILON 0.04504800699991604 / ACTION 1 / REWARD 0.1 / Q_MAX  3.9775033 / Loss  0.10871368646621704
fps: 3.777382915470747
TIMESTEP 55109 / STATE explore / EPSILON 0.04504700799991604 / ACTION 1 / REWARD 0.1 / Q_MAX  3.565585 / Loss  0.08635947108268738
fps: 3.3578286014384617
TIMESTEP 55110 / STATE explore / EPSILON 0.04504600899991604 / ACTION 1 / REWARD 0.1 / Q_MAX  5.8724713 / Loss  0.09713103622198105
fps: 3.276497947845433
TIMESTEP 55111 / STATE explore / EPSILON 0.045045009999916036 / ACTION 

TIMESTEP 55158 / STATE explore / EPSILON 0.044998056999915964 / ACTION 0 / REWARD 0.1 / Q_MAX  4.7425776 / Loss  0.1979265660047531
fps: 4.302547489747057
TIMESTEP 55159 / STATE explore / EPSILON 0.04499705799991596 / ACTION 0 / REWARD 0.1 / Q_MAX  1.0849597 / Loss  0.05098018795251846
fps: 6.3020213387744555
TIMESTEP 55160 / STATE explore / EPSILON 0.04499605899991596 / ACTION 0 / REWARD 0.1 / Q_MAX  5.97001 / Loss  0.22039666771888733
fps: 2.5058004418587703
TIMESTEP 55161 / STATE explore / EPSILON 0.04499505999991596 / ACTION 0 / REWARD 0.1 / Q_MAX  2.521646 / Loss  0.11953488737344742
fps: 4.59996578242006
TIMESTEP 55162 / STATE explore / EPSILON 0.04499406099991596 / ACTION 0 / REWARD 0.1 / Q_MAX  1.1162385 / Loss  0.22886092960834503
fps: 6.27594813216542
TIMESTEP 55163 / STATE explore / EPSILON 0.044993061999915956 / ACTION 0 / REWARD 0.1 / Q_MAX  4.7407713 / Loss  0.15867799520492554
fps: 5.885346493252828
TIMESTEP 55164 / STATE explore / EPSILON 0.044992062999915955 / ACTION 0

TIMESTEP 55212 / STATE explore / EPSILON 0.04494411099991588 / ACTION 1 / REWARD 0.1 / Q_MAX  8.817759 / Loss  0.26830798387527466
fps: 3.7531813378217915
TIMESTEP 55213 / STATE explore / EPSILON 0.04494311199991588 / ACTION 1 / REWARD 0.1 / Q_MAX  1.9632984 / Loss  0.22068804502487183
fps: 3.250179002606774
TIMESTEP 55214 / STATE explore / EPSILON 0.04494211299991588 / ACTION 1 / REWARD 0.1 / Q_MAX  8.847277 / Loss  0.3548780679702759
fps: 3.1138923431566523
TIMESTEP 55215 / STATE explore / EPSILON 0.04494111399991588 / ACTION 1 / REWARD 0.1 / Q_MAX  2.5400684 / Loss  0.24005913734436035
fps: 2.610240437124463
TIMESTEP 55216 / STATE explore / EPSILON 0.044940114999915876 / ACTION 1 / REWARD 0.1 / Q_MAX  6.252737 / Loss  0.18677979707717896
fps: 3.3914084078839273
TIMESTEP 55217 / STATE explore / EPSILON 0.044939115999915874 / ACTION 1 / REWARD 0.1 / Q_MAX  4.012027 / Loss  0.5334198474884033
fps: 6.106179255927034
TIMESTEP 55218 / STATE explore / EPSILON 0.04493811699991587 / ACTION 1

...

TIMESTEP 55530 / STATE explore / EPSILON 0.044626428999915396 / ACTION 1 / REWARD 0.1 / Q_MAX  1.16677 / Loss  0.4346312880516052
fps: 3.47163464562758
TIMESTEP 55531 / STATE explore / EPSILON 0.044625429999915395 / ACTION 0 / REWARD 0.1 / Q_MAX  -0.005561009 / Loss  0.24540627002716064
fps: 2.008709524012492
TIMESTEP 55532 / STATE explore / EPSILON 0.04462443099991539 / ACTION 1 / REWARD 0.1 / Q_MAX  3.0543613 / Loss  0.22306865453720093
fps: 4.295462133237749
TIMESTEP 55533 / STATE explore / EPSILON 0.04462343199991539 / ACTION 1 / REWARD -1 / Q_MAX  7.983039 / Loss  0.13244031369686127
fps: 2.410022299996725
TIMESTEP 55534 / STATE explore / EPSILON 0.04462243299991539 / ACTION 1 / REWARD 0.1 / Q_MAX  7.2283087 / Loss  0.14827775955200195
fps: 3.1413348432217543
TIMESTEP 55535 / STATE explore / EPSILON 0.04462143399991539 / ACTION 1 / REWARD 0.1 / Q_MAX  8.052176 / Loss  0.3970802426338196
fps: 3.042680033950192
TIMESTEP 55536 / STATE explore / EPSILON 0.04462043499991539 / ACTION 1 

TIMESTEP 55583 / STATE explore / EPSILON 0.044573481999915315 / ACTION 1 / REWARD 0.1 / Q_MAX  1.1899587 / Loss  0.034648437052965164
fps: 2.8277947900612306
TIMESTEP 55584 / STATE explore / EPSILON 0.044572482999915314 / ACTION 1 / REWARD 0.1 / Q_MAX  6.947358 / Loss  0.2842296063899994
fps: 2.1111276651432482
TIMESTEP 55585 / STATE explore / EPSILON 0.04457148399991531 / ACTION 1 / REWARD 0.1 / Q_MAX  9.124893 / Loss  0.3594048321247101
fps: 3.202362582868678
TIMESTEP 55586 / STATE explore / EPSILON 0.04457048499991531 / ACTION 1 / REWARD 0.1 / Q_MAX  2.5107172 / Loss  0.4593932330608368
fps: 2.836570851984881
TIMESTEP 55587 / STATE explore / EPSILON 0.04456948599991531 / ACTION 1 / REWARD 0.1 / Q_MAX  -0.70274127 / Loss  0.1783718317747116
fps: 5.422689609074912
TIMESTEP 55588 / STATE explore / EPSILON 0.04456848699991531 / ACTION 1 / REWARD 0.1 / Q_MAX  1.9965429 / Loss  0.22690598666667938
----------Random Action----------
fps: 6.343031186578067
TIMESTEP 55589 / STATE explore / EP

TIMESTEP 55636 / STATE explore / EPSILON 0.044520534999915234 / ACTION 0 / REWARD 0.1 / Q_MAX  4.384525 / Loss  0.21562813222408295
fps: 3.607019849211011
TIMESTEP 55637 / STATE explore / EPSILON 0.04451953599991523 / ACTION 0 / REWARD 0.1 / Q_MAX  -0.99512976 / Loss  0.17134109139442444
fps: 3.2420041770435746
TIMESTEP 55638 / STATE explore / EPSILON 0.04451853699991523 / ACTION 1 / REWARD -1 / Q_MAX  10.7241 / Loss  0.3194939196109772
fps: 3.4653617008619815
TIMESTEP 55639 / STATE explore / EPSILON 0.04451753799991523 / ACTION 1 / REWARD -1 / Q_MAX  9.933082 / Loss  0.34363090991973877
fps: 3.3551506308649888
TIMESTEP 55640 / STATE explore / EPSILON 0.04451653899991523 / ACTION 1 / REWARD 0.1 / Q_MAX  7.222258 / Loss  0.48494789004325867
fps: 3.4167033239164932
TIMESTEP 55641 / STATE explore / EPSILON 0.04451553999991523 / ACTION 1 / REWARD 0.1 / Q_MAX  5.945025 / Loss  0.1684967279434204
fps: 4.53220234589846
TIMESTEP 55642 / STATE explore / EPSILON 0.044514540999915225 / ACTION 1 /

TIMESTEP 55689 / STATE explore / EPSILON 0.044467587999915154 / ACTION 1 / REWARD 0.1 / Q_MAX  5.4866033 / Loss  0.594570517539978
fps: 2.3257860742423166
TIMESTEP 55690 / STATE explore / EPSILON 0.04446658899991515 / ACTION 1 / REWARD -1 / Q_MAX  2.7296402 / Loss  0.3607901930809021
fps: 4.8917558777872765
TIMESTEP 55691 / STATE explore / EPSILON 0.04446558999991515 / ACTION 1 / REWARD -1 / Q_MAX  3.504907 / Loss  0.1616813987493515
fps: 4.082464874756058
TIMESTEP 55692 / STATE explore / EPSILON 0.04446459099991515 / ACTION 1 / REWARD 0.1 / Q_MAX  9.977473 / Loss  0.2805810570716858
fps: 5.857516531551347
TIMESTEP 55693 / STATE explore / EPSILON 0.04446359199991515 / ACTION 1 / REWARD 0.1 / Q_MAX  8.802879 / Loss  0.1486125886440277
fps: 5.908127805769937
TIMESTEP 55694 / STATE explore / EPSILON 0.044462592999915146 / ACTION 0 / REWARD 0.1 / Q_MAX  8.047583 / Loss  0.12568672001361847
fps: 2.856373118532936
TIMESTEP 55695 / STATE explore / EPSILON 0.044461593999915144 / ACTION 1 / REW

fps: 3.297613609650285
TIMESTEP 55742 / STATE explore / EPSILON 0.04441464099991507 / ACTION 1 / REWARD 0.1 / Q_MAX  4.773201 / Loss  0.42515426874160767
fps: 4.295224597134267
TIMESTEP 55743 / STATE explore / EPSILON 0.04441364199991507 / ACTION 1 / REWARD 0.1 / Q_MAX  9.424013 / Loss  0.5291615724563599
fps: 2.126586077904273
TIMESTEP 55744 / STATE explore / EPSILON 0.04441264299991507 / ACTION 1 / REWARD 0.1 / Q_MAX  8.915629 / Loss  0.35374367237091064
fps: 3.591284274842625
TIMESTEP 55745 / STATE explore / EPSILON 0.04441164399991507 / ACTION 1 / REWARD 0.1 / Q_MAX  8.91745 / Loss  0.2842966318130493
fps: 2.8244395301292116
TIMESTEP 55746 / STATE explore / EPSILON 0.04441064499991507 / ACTION 1 / REWARD 0.1 / Q_MAX  7.001114 / Loss  0.182559072971344
----------Random Action----------
fps: 3.585622984622434
TIMESTEP 55747 / STATE explore / EPSILON 0.044409645999915065 / ACTION 1 / REWARD 0.1 / Q_MAX  5.966889 / Loss  0.23649711906909943
fps: 2.98387656723978
TIMESTEP 55748 / STATE 